In [ ]:
"""
Notebook that runs a neutral network on the adult and student performance dataset without annoymisation.
Supposed to be a benchmark for the anonimised data sets.
"""


import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

In [ ]:
"""
Load data
"""